In [1]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

# Example usage
from functools import partial

conn = LazyConnection(('www.python.org', 80))

with conn as s1:
    s1.send(b'GET /index.html HTTP/1.0\r\n')
    s1.send(b'Host: www.python.org\r\n')
    s1.send(b'\r\n')
    resp = b''.join(iter(partial(s1.recv, 8192), b''))
    print(resp)

with conn as s2:
    s2.send(b'GET /about HTTP/1.0\r\n')
    s2.send(b'Host: www.python.org\r\n')
    s2.send(b'\r\n')
    resp = b''.join(iter(partial(s2.recv, 8192), b''))
    print(resp)

b'HTTP/1.1 301 Moved Permanently\r\nConnection: close\r\nContent-Length: 0\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nAccept-Ranges: bytes\r\nDate: Mon, 27 Jan 2025 18:32:27 GMT\r\nVia: 1.1 varnish\r\nX-Served-By: cache-iad-kjyo7100033-IAD\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1738002748.766822,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains; preload\r\n\r\n'
b'HTTP/1.1 301 Moved Permanently\r\nConnection: close\r\nContent-Length: 0\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/about\r\nAccept-Ranges: bytes\r\nDate: Mon, 27 Jan 2025 18:32:27 GMT\r\nVia: 1.1 varnish\r\nX-Served-By: cache-iad-kjyo7100078-IAD\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1738002748.920913,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains; preload\r\n\r\n'


In [2]:
from contextlib import contextmanager
from socket import socket, AF_INET, SOCK_STREAM

@contextmanager
def lazy_connection(address, family=AF_INET, type=SOCK_STREAM):
    sock = socket(family, type)
    sock.connect(address)
    try:
        yield sock
    finally:
        sock.close()

# Example usage
with lazy_connection(('www.python.org', 80)) as s:
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    print(resp)

b'HTTP/1.1 301 Moved Permanently\r\nConnection: close\r\nContent-Length: 0\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nAccept-Ranges: bytes\r\nDate: Mon, 27 Jan 2025 18:33:28 GMT\r\nVia: 1.1 varnish\r\nX-Served-By: cache-iad-kjyo7100157-IAD\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1738002808.063369,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains; preload\r\n\r\n'
